In [2]:
from dicom_file import dicom_2_array

from random import random
import pandas as pd
import numpy as np

import torch, torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Sampler

In [3]:
df = pd.read_csv("final_df.csv")

df.loc[df['Contrast']==True,['Contrast']] = 1
df.loc[df['Contrast']==False,['Contrast']] = 0

df['Contrast'] = df['Contrast'].astype('int')

train_images = df['dicom_name'].values
train_labels = df['Contrast'].values

In [19]:
class CustomDataset(Dataset):
    def __init__(self,train_images,train_labels):
    
        self.x_data = [dicom_2_array('dicom_dir/' + path) for path in train_images]
        self.y_data = [[i] for i in train_labels]

    def __len__(self):

        return len(self.x_data)

    def __getitem__(self, idx):

        x = torch.FloatTensor(self.x_data[idx])
        y = torch.LongTensor(self.y_data[idx])

        return x,y

In [20]:
class customsampler(Sampler):

    def __init__(self, data, positive_num, negative_num):

        self.data = data
        self.total = len(self.data)
        self.positive_num = positive_num
        self.negative_num = negative_num

        self.batch_size = self.positive_num + self.negative_num
        self.iter = self.total // self.batch_size

        self.positive_list = len([i for i, predict in enumerate(self.data) if predict == 1 ])

        self.idx = list(range(self.total)) 

    def __iter__(self):
        batch = list()

        for i in range(self.iter):

            temp = np.concatenate((random.sample(self.idx[:self.positive_list], self.positive_num),
                                random.sample(self.idx[self.positive_list:], self.negative_num)))

            random.shuffle(temp)
            batch.extend(temp)
            
        return iter(batch)

    def __len__(self) -> int:
        return self.iter * self.batch_size

    def get_num_batch(self) -> int:
        return self.iter


In [23]:
dataset = CustomDataset(train_images, train_labels)
sampler = customsampler(train_labels, 8,8)
dataloader = DataLoader(dataset, batch_size=16,sampler=sampler)

total_batch = len(dataloader)

In [24]:
for num, data in enumerate(dataloader):
    image, label = data
    print(image,label)
    if num ==0 : break

tensor([[[-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04],
         [-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04],
         [-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04],
         ...,
         [-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04],
         [-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04],
         [-3.3792e+04, -3.3792e+04, -3.3792e+04,  ..., -3.3792e+04,
          -3.3792e+04, -3.3792e+04]],

        [[-6.1073e+01, -6.1073e+01, -6.1073e+01,  ..., -6.1073e+01,
          -6.1073e+01, -6.1073e+01],
         [-6.1073e+01, -6.1073e+01, -6.1073e+01,  ..., -6.1073e+01,
          -6.1073e+01, -6.1073e+01],
         [-6.1073e+01, -6.1073e+01, -6.1073e+01,  ..., -6.1073e+01,
          -6.1073e+01, -6.1073e+01],
         ...,
         [-6.1073e+01, -6